In [30]:
from pathlib import Path
import re
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

IN_PARQUET = "cdr_keep_after_attn.parquet"
OUT_DIR = Path("analysis_out")
OUT_DIR.mkdir(exist_ok=True)

# -----------------------------
# Load
# -----------------------------
df = pd.read_parquet(IN_PARQUET)

# --- FIX: build proper dem_age_band from "Age " column (Dutch labels) ---
if "Age " in df.columns:
    def _norm_age_band(x):
        if pd.isna(x): return np.nan
        s = str(x).strip()
        nums = re.findall(r"\d+", s)  # e.g., "18 - 24 jaar" -> ["18","24"]
        if len(nums) >= 2:
            a, b = nums[0], nums[1]
            return f"{a}–{b}"  # en dash
        return np.nan

    bands = df["Age "].map(_norm_age_band)

    # Your survey uses exactly these bands; enforce them strictly
    allowed = ["18–24", "25–34", "35–44", "45–54", "55–65"]
    bands = bands.where(bands.isin(allowed))  # anything else (e.g., "3") -> NaN
    df["dem_age_band"] = pd.Categorical(bands, categories=allowed, ordered=True)

    # Keep raw for reference
    if "dem_age_band_raw" not in df.columns:
        df["dem_age_band_raw"] = df["Age "]

# -----------------------------
# Helpers (formatting + stats)
# -----------------------------
def cronbach_alpha(df, cols):
    cols = [c for c in cols if c in df.columns]
    if len(cols) < 2:
        return np.nan, 0
    sub = df[cols].apply(pd.to_numeric, errors="coerce").dropna()
    n, k = sub.shape
    if n < 2 or k < 2:
        return np.nan, n
    var_sum = sub.var(ddof=1).sum()
    total_var = sub.sum(axis=1).var(ddof=1)
    if total_var <= 0:
        return np.nan, n
    alpha = (k/(k-1))*(1 - var_sum/total_var)
    return float(alpha), n

def _fmt_val(v):
    if pd.isna(v):
        return ""
    if isinstance(v, (int, np.integer)):
        return f"{int(v):,}".replace(",", " ")
    if isinstance(v, (float, np.floating)):
        if float(v).is_integer():
            return f"{int(v):,}".replace(",", " ")
        s = f"{v:.3f}".rstrip("0").rstrip(".")
        return s
    return str(v)

def md_table(df_md):
    """Pretty Markdown table: left-align text, right-align numbers, neat formatting."""
    df_md = df_md.copy()
    for c in df_md.columns:
        df_md[c] = df_md[c].map(_fmt_val)
    cols = list(df_md.columns)
    lines = []
    lines.append("| " + " | ".join(cols) + " |")
    aligns = []
    for i, c in enumerate(cols):
        try:
            if pd.to_numeric(df_md[c].str.replace(" ", ""), errors="coerce").notna().mean() > 0.6 and i != 0:
                aligns.append("--:")
            else:
                aligns.append(":--")
        except Exception:
            aligns.append(":--")
    lines.append("|" + "|".join(aligns) + "|")
    for _, r in df_md.iterrows():
        lines.append("| " + " | ".join("" if pd.isna(v) else str(v) for v in r.values) + " |")
    return "\n".join(lines)

def save_markdown(name, content):
    p = OUT_DIR / name
    p.write_text(content, encoding="utf-8")
    print(f"[written] {p}")
    return p

def present(cols):
    return [c for c in cols if c in df.columns]

def vc_pct(series, topn=20):
    """Value counts with %; if categorical, keep category order."""
    if hasattr(series, "cat"):
        vc = series.value_counts(dropna=False, sort=False)
    else:
        vc = series.value_counts(dropna=False)
    pct = (vc / vc.sum() * 100).round(1)
    out = pd.DataFrame({"n": vc, "%": pct})
    return out.head(topn)

def _repr_cols(n=60):
    return [repr(c) for c in list(df.columns)[:n]]

def _find_col(df, *cands):
    for c in cands:
        if c in df.columns: return c
        c2 = c.strip()
        if c2 in df.columns: return c2
    lower_map = {str(k).strip().lower(): k for k in df.columns}
    for c in cands:
        key = c.strip().lower()
        if key in lower_map: return lower_map[key]
    for c in cands:
        key = c.strip().lower()
        for k, orig in lower_map.items():
            if key in k:
                return orig
    return None

# -----------------------------
# Age banding sanity check (keep only allowed; already enforced above)
# -----------------------------
def normalize_age_bands():
    allowed = pd.Index(["18–24","25–34","35–44","45–54","55–65"])
    if "dem_age_band" in df.columns:
        tmp = df["dem_age_band"].astype(str)
        df.loc[~tmp.isin(allowed.astype(str)), "dem_age_band"] = np.nan
        df["dem_age_band"] = pd.Categorical(df["dem_age_band"], categories=allowed, ordered=True)

normalize_age_bands()

# -----------------------------
# Ensure other demogs exist (gender/education/employment/LR)
# -----------------------------
def ensure_other_demogs():
    raw_gend = _find_col(df, "Gender", "Gender ", "Geslacht")
    raw_edu  = _find_col(df, "Education", "Education ", "Opleiding")
    raw_work = _find_col(df, "Work_status", "Work status", "Werkstatus")
    raw_pol  = _find_col(df, "Pol_orientation", "Pol orientation", "Politieke", "Politiek", "L–R", "LR", "LeftRight")

    if "dem_gender" not in df.columns and raw_gend is not None:
        df["dem_gender"] = df[raw_gend].astype("category")
    if "dem_education" not in df.columns and raw_edu is not None:
        df["dem_education"] = df[raw_edu].astype("category")
    if "dem_employment" not in df.columns and raw_work is not None:
        df["dem_employment"] = df[raw_work].astype("category")

    if "dem_political_lr" not in df.columns and raw_pol is not None:
        s = pd.to_numeric(df[raw_pol], errors="coerce")
        if s.notna().sum() >= 10:
            df["dem_political_lr"] = s
        else:
            def _first_int(x):
                if pd.isna(x): return np.nan
                m = re.search(r'[-+]?\d+', str(x))
                return float(m.group(0)) if m else np.nan
            df["dem_political_lr"] = df[raw_pol].map(_first_int)

ensure_other_demogs()

# Make demogs categorical where appropriate
for cat_col in ["dem_age_band", "dem_gender", "dem_education", "dem_employment"]:
    if cat_col in df.columns and (df[cat_col].dtype == object or not str(df[cat_col].dtype).startswith("category")):
        df[cat_col] = df[cat_col].astype("category")

# -----------------------------
# Column sets (as created in your build step)
# -----------------------------
CDR_ENV = present([f"CDR attributions _{i}" for i in range(1,4)])
CDR_SOC = present([f"CDR attributions _{i}" for i in range(4,13)])
CDR_GOV = present([f"CDR attributions _{i}" for i in range(13,21)])
CDR_TOTAL = CDR_ENV + CDR_SOC + CDR_GOV

# Composite columns from build
DV_TOTAL = "CDR_total_mean"
DV_ENV   = "CDR_env_mean"
DV_SOC   = "CDR_soc_mean"
DV_GOV   = "CDR_gov_mean"

PREDICTORS = present(["env_consc", "soc_consc", "trust_mean", "digiQ_prop", "digiQ_total"])
DEMOGS = present(["dem_age_band", "dem_gender", "dem_education", "dem_employment", "dem_political_lr"])

# RQ1 blocks
RQ1_ENV = present([f"responsibility attri_{i}" for i in range(1,9)])
RQ1_SOC = present([f"resp_attribut_societ_{i}" for i in range(1,9)])
RQ1_GOV = present([f"resp_attribut_regula_{i}" for i in range(1,9)])

# -----------------------------
# 1) Sample overview (N, demogs)
# -----------------------------
N = len(df)

demo_blocks = []
if "dem_age_band" in df:    demo_blocks.append(("Leeftijd", vc_pct(df["dem_age_band"])))
if "dem_gender" in df:      demo_blocks.append(("Gender",   vc_pct(df["dem_gender"])))
if "dem_education" in df:   demo_blocks.append(("Opleiding",vc_pct(df["dem_education"])))
if "dem_employment" in df:  demo_blocks.append(("Werkstatus",vc_pct(df["dem_employment"])))
if "dem_political_lr" in df:
    demo_blocks.append(("Politiek L–R", df["dem_political_lr"].describe()[["count","mean","std","min","25%","50%","75%","max"]].to_frame().T.round(2)))

sample_lines = []
sample_lines.append("# 📊 Sample overview\n")
sample_lines.append(md_table(pd.DataFrame([{"Metric":"Analytic sample (kept)", "N": N}])))
sample_lines.append("\n## 👥 Demographics")

if demo_blocks:
    for title, tbl in demo_blocks:
        sample_lines.append(f"\n### {title}\n")
        if isinstance(tbl, pd.DataFrame):
            sample_lines.append(md_table(tbl.reset_index().rename(columns={"index": title})))
        else:
            sample_lines.append(str(tbl))
else:
    sample_lines.append("\n_No demographics found in dataset._")
    sample_lines.append("\n<details><summary>Debug: first headers</summary>\n\n```\n" + "\n".join(_repr_cols(40)) + "\n```\n</details>")

save_markdown("01_sample_overview.md", "\n".join(sample_lines))

# -----------------------------
# 2) Reliability (α)
# -----------------------------
alpha_rows = []
for label, cols in [("Environmental", CDR_ENV), ("Social", CDR_SOC), ("Governance", CDR_GOV)]:
    a, n = cronbach_alpha(df, cols)
    alpha_rows.append({"Domein": label, "k": len(cols), "N_alpha": n, "Cronbach_α": (None if pd.isna(a) else round(a,3))})
alpha_df = pd.DataFrame(alpha_rows)
alpha_md = "# 5️⃣ Betrouwbaarheid — Cronbach’s α\n\n" + md_table(alpha_df)
save_markdown("02_reliability.md", alpha_md)

# -----------------------------
# 3) Descriptives
# -----------------------------
desc_cols = present([DV_TOTAL, DV_ENV, DV_SOC, DV_GOV] + PREDICTORS + ["CDR_total_mean"])
if desc_cols:
    desc = df[desc_cols].apply(pd.to_numeric, errors="coerce").describe().T
    desc = desc.rename(columns={"mean":"Mean","std":"SD","min":"Min","25%":"Q25","50%":"Median","75%":"Q75","max":"Max"})
    desc_md = "# 📈 Descriptives\n\n" + md_table(desc[["count","Mean","SD","Min","Q25","Median","Q75","Max"]].round(3).reset_index().rename(columns={"index":"Variable"}))
else:
    desc_md = "# 📈 Descriptives\n\n_No eligible numeric variables found._"
save_markdown("03_descriptives.md", desc_md)

# -----------------------------
# 4) Correlations
# -----------------------------
corr_cols = present([DV_TOTAL, DV_ENV, DV_SOC, DV_GOV, "env_consc", "soc_consc", "trust_mean", "digiQ_prop", "dem_political_lr"])
if len(corr_cols) >= 2:
    corr_df = df[corr_cols].apply(pd.to_numeric, errors="coerce").corr(min_periods=50)
    corr_path = OUT_DIR / "04_correlations.csv"
    corr_df.round(3).to_csv(corr_path)
    print(f"[written] {corr_path}")
    corr_preview = corr_df.round(3).reset_index().rename(columns={"index":"Var"})
    corr_md = "# 🔗 Correlations (Pearson)\n\n" + md_table(corr_preview.iloc[: min(15, len(corr_preview)), : min(8, len(corr_preview.columns))])
else:
    corr_md = "# 🔗 Correlations (Pearson)\n\n_Not enough variables for a correlation matrix._"
save_markdown("04_correlations.md", corr_md)

# -----------------------------
# 5) Regressions (OLS, HC3 SE)
# -----------------------------
def build_formula(dv):
    base = ["env_consc","soc_consc","trust_mean","digiQ_prop","dem_political_lr"]
    X = [c for c in base if c in df.columns and df[c].notna().sum() > 1]
    cats_raw = [c for c in ["dem_age_band","dem_gender","dem_education","dem_employment"]
                if c in df.columns and df[c].dropna().nunique() > 1]
    cats = [f"C({c})" for c in cats_raw]
    nums = [c for c in ["dem_political_lr"] if c in df.columns and df[c].notna().sum() > 1]
    terms = X + cats + [n for n in nums if n not in X]
    if not terms:
        return None
    return f"{dv} ~ " + " + ".join(terms)

def tidy_ols(model, model_name):
    params = model.params
    bse    = model.bse
    if hasattr(model, "tvalues"):
        stat = model.tvalues
    elif hasattr(model, "zvalues"):
        stat = model.zvalues
    else:
        stat = None
    ci     = model.conf_int()
    out = pd.DataFrame({
        "term": params.index,
        "coef": params.values,
        "se": bse.values,
        "stat": (stat.values if stat is not None else np.full_like(params.values, np.nan, dtype=float)),
        "p": model.pvalues.values,
        "ci_low": ci[0].values,
        "ci_high": ci[1].values,
    })
    out["model"] = model_name
    return out

def run_model(dv, name):
    formula = build_formula(dv)
    if not formula:
        return None, None
    try:
        model = smf.ols(formula, data=df).fit(cov_type="HC3")
    except Exception as e:
        note = f"# ⚠️ Regression skipped: {name}\n\n**Formula:** `{formula}`\n\n```\n{e}\n```"
        save_markdown(f"05_regressions_{name.replace(' ','_')}_ERROR.md", note)
        return None, None
    table = tidy_ols(model, name)
    return model, table

reg_tables = []
models_run = []
for dv, name in [(DV_TOTAL,"CDR Total"), (DV_ENV,"CDR Env"), (DV_SOC,"CDR Soc"), (DV_GOV,"CDR Gov")]:
    if dv in df.columns:
        m, t = run_model(dv, name)
        if t is not None:
            reg_tables.append(t)
            models_run.append((name, m))

def _stars(p):
    return "***" if p < 0.001 else "**" if p < 0.01 else "*" if p < 0.05 else ""

if reg_tables:
    reg_all = pd.concat(reg_tables, ignore_index=True)
    reg_all_out = reg_all.copy()
    reg_all_out["sig"] = reg_all_out["p"].apply(_stars)
    reg_all_out[["coef","se","stat","p","ci_low","ci_high"]] = reg_all_out[["coef","se","stat","p","ci_low","ci_high"]].round(4)
    reg_all_out.to_csv(OUT_DIR / "05_regressions.csv", index=False)
    reg_md = "# 📉 OLS regressions (HC3 SE)\n\n" + md_table(
        reg_all_out[["model","term","coef","se","stat","p","sig","ci_low","ci_high"]]
        .rename(columns={"coef":"Coef","se":"SE","stat":"t/z","p":"p","ci_low":"CI_low","ci_high":"CI_high"})
    )
    save_markdown("05_regressions.md", reg_md)
else:
    save_markdown("05_regressions.md", "# 📉 OLS regressions\n\n_No eligible models (missing predictors/DVs)._")

# -----------------------------
# 6) Responsibility attribution (RQ1)
# -----------------------------
def summarize_block(cols, title):
    if not cols: return f"## {title}\n\n_Geen kolommen gevonden._"
    sub = df[cols].apply(pd.to_numeric, errors="coerce")
    means = sub.mean(axis=0).sort_values(ascending=False).round(3)
    tbl = pd.DataFrame({"actor_item": means.index, "mean": means.values})
    tbl["rank"] = np.arange(1, len(tbl)+1)
    return f"## {title}\n\n" + md_table(tbl)

rq1_parts = []
rq1_parts.append(summarize_block(RQ1_ENV, "RQ1 — Environment: perceived responsibility (higher = more responsibility)"))
rq1_parts.append(summarize_block(RQ1_SOC, "RQ1 — Social: perceived responsibility"))
rq1_parts.append(summarize_block(RQ1_GOV, "RQ1 — Governance/Ethical use: perceived responsibility"))

rq1_md = "# 🧭 Responsibility Attributions\n\n" + "\n\n".join(rq1_parts)
save_markdown("06_responsibility_attribution.md", rq1_md)

# -----------------------------
# 7) Mini preregistration scaffold
# -----------------------------
alpha_md_inline = md_table(pd.DataFrame([
    {"Domein":"Environmental","k":len(CDR_ENV),"N_alpha":""},
    {"Domein":"Social","k":len(CDR_SOC),"N_alpha":""},
    {"Domein":"Governance","k":len(CDR_GOV),"N_alpha":""},
]).assign(**{"Cronbach_α":""}))

prereg = f"""
# 📝 Preregistration scaffold (draft)

## Outcomes
- **CDR_total_mean** (primary), **CDR_env_mean**, **CDR_soc_mean**, **CDR_gov_mean** (secondary).

## Predictors & Covariates
- Predictors: env_consc, soc_consc, trust_mean, digiQ_prop.
- Covariates: dem_age_band, dem_gender, dem_education, dem_employment, dem_political_lr (as available).

## Reliability
{alpha_md_inline}

## Models
- **M1 (Primary):** CDR_total_mean ~ z(env_consc) + z(soc_consc) + z(trust_mean) + z(digiQ_prop) + demographics.
- **M2–M4:** Same predictors with CDR_env_mean, CDR_soc_mean, CDR_gov_mean.

(Where z(x) denotes standardization; in the quick results above we reported unstandardized β with HC3 SE.)

## Missing data
- Listwise deletion within each model. No imputation for prereg primary; sensitivity analyses optional.

## Responsibility attribution (RQ1)
- Report actor-level means within each domain; interpret rank order patterns.
"""
save_markdown("07_prereg_scaffold.md", prereg)

# -----------------------------
# 8) Stitch single report in prereg order with TOC
# -----------------------------
def make_toc(md: str) -> str:
    lines = []
    for line in md.splitlines():
        m = re.match(r"^(#{1,3})\s+(.*)", line.strip())
        if m:
            level = len(m.group(1))
            title = m.group(2).strip()
            anchor = re.sub(r"[^\w\- ]", "", title).strip().lower().replace(" ", "-")
            lines.append(f"{'  '*(level-1)}- [{title}](#{anchor})")
    return "\n".join(lines)

ORDER = [
    "01_sample_overview.md",
    "02_reliability.md",
    "03_descriptives.md",
    "04_correlations.md",
    "05_regressions.md",
    "06_responsibility_attribution.md",
    "07_prereg_scaffold.md",
]

parts = []
for fname in ORDER:
    p = OUT_DIR / fname
    if p.exists():
        parts.append(f"\n<!-- ⤷ {fname} -->\n")
        parts.append(p.read_text(encoding="utf-8"))
    else:
        parts.append(f"\n> _Missing: {fname}_\n")

body = "\n".join(parts).strip()
report = f"""# CDR — Analysis Report (prereg order)

_This document stitches the individual outputs into a single narrative, matching our preregistration structure._

## Table of Contents
{make_toc(body)}

---

{body}
"""
(OUT_DIR / "00_REPORT.md").write_text(report, encoding="utf-8")
print(f"[written] {OUT_DIR / '00_REPORT.md'}")

print("\nDone. Open the 'analysis_out' folder for the Markdown + CSV outputs.\n")


[written] analysis_out/01_sample_overview.md
[written] analysis_out/02_reliability.md
[written] analysis_out/03_descriptives.md
[written] analysis_out/04_correlations.csv
[written] analysis_out/04_correlations.md
[written] analysis_out/05_regressions.md
[written] analysis_out/06_responsibility_attribution.md
[written] analysis_out/07_prereg_scaffold.md
[written] analysis_out/00_REPORT.md

Done. Open the 'analysis_out' folder for the Markdown + CSV outputs.



In [31]:
# Render the analysis outputs directly in this notebook

from pathlib import Path
from IPython.display import Markdown, display
import pandas as pd

OUT_DIR = Path("analysis_out")

# 1) Show the stitched report first (single document, prereg order)
report = OUT_DIR / "00_REPORT.md"
if report.exists():
    display(Markdown(f"# 📓 Notebook View — Full Report\n\n_This is the stitched version (same as `{report.name}`), rendered inline for quick reading._"))
    display(Markdown(report.read_text(encoding="utf-8")))
else:
    display(Markdown("**Note:** `00_REPORT.md` not found. I'll render the individual sections below."))

# 2) Also render the individual sections (useful for quick navigation)
ORDER = [
    "01_sample_overview.md",
    "02_reliability.md",
    "03_descriptives.md",
    "04_correlations.md",
    "05_regressions.md",
    "06_responsibility_attribution.md",
    "07_prereg_scaffold.md",
]

display(Markdown("---"))
display(Markdown("# 🔎 Individual Sections"))
for fname in ORDER:
    p = OUT_DIR / fname
    if p.exists():
        display(Markdown(f"---\n\n### ⤷ {fname}\n"))
        display(Markdown(p.read_text(encoding="utf-8")))
    else:
        display(Markdown(f"> _Missing: {fname}_"))

# 3) (Optional) Show the correlations dataframe inline for quick scanning
corr_csv = OUT_DIR / "04_correlations.csv"
if corr_csv.exists():
    df_corr = pd.read_csv(corr_csv, index_col=0)
    display(Markdown("### 📊 Correlations (DataFrame view)"))
    import caas_jupyter_tools
    caas_jupyter_tools.display_dataframe_to_user("Correlations (04_correlations.csv)", df_corr)


# 📓 Notebook View — Full Report

_This is the stitched version (same as `00_REPORT.md`), rendered inline for quick reading._

# CDR — Analysis Report (prereg order)

_This document stitches the individual outputs into a single narrative, matching our preregistration structure._

## Table of Contents
- [📊 Sample overview](#sample-overview)
  - [👥 Demographics](#demographics)
    - [Leeftijd](#leeftijd)
    - [Gender](#gender)
    - [Opleiding](#opleiding)
    - [Werkstatus](#werkstatus)
    - [Politiek L–R](#politiek-lr)
- [5️⃣ Betrouwbaarheid — Cronbach’s α](#5-betrouwbaarheid--cronbachs-α)
- [📈 Descriptives](#descriptives)
- [🔗 Correlations (Pearson)](#correlations-pearson)
- [📉 OLS regressions (HC3 SE)](#ols-regressions-hc3-se)
- [🧭 Responsibility Attributions](#responsibility-attributions)
  - [RQ1 — Environment: perceived responsibility (higher = more responsibility)](#rq1--environment-perceived-responsibility-higher--more-responsibility)
  - [RQ1 — Social: perceived responsibility](#rq1--social-perceived-responsibility)
  - [RQ1 — Governance/Ethical use: perceived responsibility](#rq1--governanceethical-use-perceived-responsibility)
- [📝 Preregistration scaffold (draft)](#preregistration-scaffold-draft)
  - [Outcomes](#outcomes)
  - [Predictors & Covariates](#predictors--covariates)
  - [Reliability](#reliability)
  - [Models](#models)
  - [Missing data](#missing-data)
  - [Responsibility attribution (RQ1)](#responsibility-attribution-rq1)

---

<!-- ⤷ 01_sample_overview.md -->

# 📊 Sample overview

| Metric | N |
|:--|--:|
| Analytic sample (kept) | 2 246 |

## 👥 Demographics

### Leeftijd

| dem_age_band | n | % |
|:--|--:|--:|
| 18–24 | 239 | 10.6 |
| 25–34 | 497 | 22.1 |
| 35–44 | 447 | 19.9 |
| 45–54 | 520 | 23.2 |
| 55–65 | 540 | 24 |
|  | 3 | 0.1 |

### Gender

| dem_gender | n | % |
|:--|--:|--:|
| Man | 1 095 | 48.8 |
| Non-binair | 7 | 0.3 |
| Vrouw | 1 140 | 50.8 |
| Zeg ik liever niet | 4 | 0.2 |

### Opleiding

| dem_education | n | % |
|:--|--:|--:|
| Basisschool, lager onderwijs | 41 | 1.8 |
| Gepromoveerd (dr.) | 25 | 1.1 |
| Hoger algemeen en voorbereidend wetenschappelijk onderwijs (HAVO, VWO, HBS, Gymnasium, Atheneum) | 159 | 7.1 |
| Hoger beroepsonderwijs (HBO, HTS, HEAO, Kweekschool, Sociale of Pedagogische Academie) | 635 | 28.3 |
| Lager beroepsonderwijs (VMBO-B, VMBO-K, praktijkonderwijs, VGLO, LAVO, LTS, Huishoudschool) | 176 | 7.8 |
| Middelbaar algemeen beroepsonderwijs (MBO, MTS) | 389 | 17.3 |
| Middelbaar algemeen voortgezet onderwijs (VMBO-T, MAVO, MULO, 3-jarige HBS) | 241 | 10.7 |
| Universitaire Bachelor (BA, BSc) | 264 | 11.8 |
| Universitaire Master (drs., MA, MSc, mr., ir) | 316 | 14.1 |

### Werkstatus

| dem_employment | n | % |
|:--|--:|--:|
| Anders, namelijk: | 59 | 2.6 |
| Ik ben arbeidsongeschikt | 195 | 8.7 |
| Ik ben met (vervroegd) pensioen | 48 | 2.1 |
| Ik ben student/scholier   | 96 | 4.3 |
| Ik ben uitzendkracht / oproepkracht / invalkracht | 23 | 1 |
| Ik ben werkloos | 117 | 5.2 |
| Ik ben zelfstandig ondernemer / zzp’er | 198 | 8.8 |
| Ik werk in loondienst (vast of tijdelijk contract) | 1 510 | 67.2 |

### Politiek L–R

| Politiek L–R | count | mean | std | min | 25% | 50% | 75% | max |
|:--|--:|--:|--:|--:|--:|--:|--:|--:|
| dem_political_lr | 2 241 | 5.78 | 2.21 | 1 | 4 | 6 | 7 | 10 |

<!-- ⤷ 02_reliability.md -->

# 5️⃣ Betrouwbaarheid — Cronbach’s α

| Domein | k | N_alpha | Cronbach_α |
|:--|--:|--:|--:|
| Environmental | 3 | 2 214 | 0.832 |
| Social | 9 | 2 214 | 0.914 |
| Governance | 8 | 2 214 | 0.944 |

<!-- ⤷ 03_descriptives.md -->

# 📈 Descriptives

| Variable | count | Mean | SD | Min | Q25 | Median | Q75 | Max |
|:--|--:|--:|--:|--:|--:|--:|--:|--:|
| CDR_total_mean | 2 214 | 5.463 | 1.086 | 1 | 4.55 | 5.6 | 6.4 | 7 |
| CDR_env_mean | 2 214 | 5.334 | 1.242 | 1 | 4.333 | 5.333 | 6.333 | 7 |
| CDR_soc_mean | 2 214 | 5.364 | 1.095 | 1 | 4.444 | 5.444 | 6.222 | 7 |
| CDR_gov_mean | 2 214 | 5.621 | 1.168 | 1 | 4.625 | 5.875 | 6.75 | 7 |
| digiQ_prop | 2 199 | 0.669 | 0.297 | 0 | 0.5 | 0.75 | 0.875 | 1 |
| digiQ_total | 2 246 | 5.208 | 2.48 | 0 | 4 | 6 | 7 | 8 |
| CDR_total_mean | 2 214 | 5.463 | 1.086 | 1 | 4.55 | 5.6 | 6.4 | 7 |

<!-- ⤷ 04_correlations.md -->

# 🔗 Correlations (Pearson)

| Var | CDR_total_mean | CDR_env_mean | CDR_soc_mean | CDR_gov_mean | digiQ_prop | dem_political_lr |
|:--|--:|--:|--:|--:|--:|--:|
| CDR_total_mean | 1 | 0.857 | 0.969 | 0.961 | 0.476 | -0.097 |
| CDR_env_mean | 0.857 | 1 | 0.787 | 0.762 | 0.39 | -0.126 |
| CDR_soc_mean | 0.969 | 0.787 | 1 | 0.883 | 0.432 | -0.084 |
| CDR_gov_mean | 0.961 | 0.762 | 0.883 | 1 | 0.495 | -0.087 |
| digiQ_prop | 0.476 | 0.39 | 0.432 | 0.495 | 1 | -0.134 |
| dem_political_lr | -0.097 | -0.126 | -0.084 | -0.087 | -0.134 | 1 |

<!-- ⤷ 05_regressions.md -->

# 📉 OLS regressions (HC3 SE)

| model | term | Coef | SE | t/z | p | sig | CI_low | CI_high |
|:--|:--|--:|--:|--:|--:|:--|--:|--:|
| CDR Total | Intercept | 3.877 | 0.223 | 17.378 | 0 | *** | 3.439 | 4.314 |
| CDR Total | C(dem_age_band)[T.25–34] | 0.166 | 0.076 | 2.181 | 0.029 | * | 0.017 | 0.316 |
| CDR Total | C(dem_age_band)[T.35–44] | 0.299 | 0.079 | 3.769 | 0 | *** | 0.143 | 0.454 |
| CDR Total | C(dem_age_band)[T.45–54] | 0.307 | 0.08 | 3.852 | 0 | *** | 0.151 | 0.463 |
| CDR Total | C(dem_age_band)[T.55–65] | 0.614 | 0.081 | 7.62 | 0 | *** | 0.456 | 0.771 |
| CDR Total | C(dem_gender)[T.Non-binair] | 0.49 | 0.465 | 1.055 | 0.291 |  | -0.42 | 1.401 |
| CDR Total | C(dem_gender)[T.Vrouw] | 0.166 | 0.041 | 4.012 | 0 | *** | 0.085 | 0.247 |
| CDR Total | C(dem_gender)[T.Zeg ik liever niet] | 0.345 | 0.505 | 0.683 | 0.494 |  | -0.645 | 1.335 |
| CDR Total | C(dem_education)[T.Gepromoveerd (dr.)] | 0.555 | 0.232 | 2.393 | 0.017 | * | 0.1 | 1.01 |
| CDR Total | C(dem_education)[T.Hoger algemeen en voorbereidend wetenschappelijk onderwijs (HAVO, VWO, HBS, Gymnasium, Atheneum)] | 0.237 | 0.188 | 1.262 | 0.207 |  | -0.131 | 0.606 |
| CDR Total | C(dem_education)[T.Hoger beroepsonderwijs (HBO, HTS, HEAO, Kweekschool, Sociale of Pedagogische Academie)] | 0.306 | 0.174 | 1.754 | 0.08 |  | -0.036 | 0.647 |
| CDR Total | C(dem_education)[T.Lager beroepsonderwijs (VMBO-B, VMBO-K, praktijkonderwijs, VGLO, LAVO, LTS, Huishoudschool)] | 0.119 | 0.186 | 0.643 | 0.52 |  | -0.245 | 0.484 |
| CDR Total | C(dem_education)[T.Middelbaar algemeen beroepsonderwijs (MBO, MTS)] | 0.174 | 0.175 | 0.995 | 0.32 |  | -0.169 | 0.518 |
| CDR Total | C(dem_education)[T.Middelbaar algemeen voortgezet onderwijs (VMBO-T, MAVO, MULO, 3-jarige HBS)] | 0.064 | 0.18 | 0.356 | 0.722 |  | -0.289 | 0.417 |
| CDR Total | C(dem_education)[T.Universitaire Bachelor (BA, BSc)] | 0.423 | 0.179 | 2.369 | 0.018 | * | 0.073 | 0.773 |
| CDR Total | C(dem_education)[T.Universitaire Master (drs., MA, MSc, mr., ir)] | 0.312 | 0.18 | 1.736 | 0.083 |  | -0.04 | 0.665 |
| CDR Total | C(dem_employment)[T.Ik ben arbeidsongeschikt] | 0.005 | 0.149 | 0.032 | 0.974 |  | -0.287 | 0.296 |
| CDR Total | C(dem_employment)[T.Ik ben met (vervroegd) pensioen] | -0.079 | 0.194 | -0.409 | 0.683 |  | -0.459 | 0.301 |
| CDR Total | C(dem_employment)[T.Ik ben student/scholier  ] | 0.062 | 0.17 | 0.366 | 0.714 |  | -0.272 | 0.397 |
| CDR Total | C(dem_employment)[T.Ik ben uitzendkracht / oproepkracht / invalkracht] | 0.061 | 0.207 | 0.296 | 0.767 |  | -0.345 | 0.468 |
| CDR Total | C(dem_employment)[T.Ik ben werkloos] | 0.16 | 0.157 | 1.015 | 0.31 |  | -0.149 | 0.468 |
| CDR Total | C(dem_employment)[T.Ik ben zelfstandig ondernemer / zzp’er] | -0.113 | 0.145 | -0.78 | 0.436 |  | -0.397 | 0.171 |
| CDR Total | C(dem_employment)[T.Ik werk in loondienst (vast of tijdelijk contract)] | -0.104 | 0.129 | -0.806 | 0.42 |  | -0.357 | 0.149 |
| CDR Total | digiQ_prop | 1.591 | 0.066 | 24.078 | 0 | *** | 1.461 | 1.72 |
| CDR Total | dem_political_lr | -0.011 | 0.009 | -1.13 | 0.259 |  | -0.029 | 0.008 |
| CDR Env | Intercept | 3.821 | 0.276 | 13.847 | 0 | *** | 3.28 | 4.362 |
| CDR Env | C(dem_age_band)[T.25–34] | 0.179 | 0.101 | 1.779 | 0.075 |  | -0.018 | 0.377 |
| CDR Env | C(dem_age_band)[T.35–44] | 0.345 | 0.103 | 3.364 | 0.001 | *** | 0.144 | 0.546 |
| CDR Env | C(dem_age_band)[T.45–54] | 0.37 | 0.103 | 3.59 | 0 | *** | 0.168 | 0.573 |
| CDR Env | C(dem_age_band)[T.55–65] | 0.615 | 0.106 | 5.775 | 0 | *** | 0.406 | 0.823 |
| CDR Env | C(dem_gender)[T.Non-binair] | 0.349 | 0.51 | 0.685 | 0.493 |  | -0.65 | 1.348 |
| CDR Env | C(dem_gender)[T.Vrouw] | 0.148 | 0.05 | 3.001 | 0.003 | ** | 0.051 | 0.245 |
| CDR Env | C(dem_gender)[T.Zeg ik liever niet] | 0.589 | 0.618 | 0.953 | 0.341 |  | -0.622 | 1.8 |
| CDR Env | C(dem_education)[T.Gepromoveerd (dr.)] | 0.69 | 0.227 | 3.042 | 0.002 | ** | 0.245 | 1.135 |
| CDR Env | C(dem_education)[T.Hoger algemeen en voorbereidend wetenschappelijk onderwijs (HAVO, VWO, HBS, Gymnasium, Atheneum)] | 0.066 | 0.21 | 0.313 | 0.755 |  | -0.347 | 0.478 |
| CDR Env | C(dem_education)[T.Hoger beroepsonderwijs (HBO, HTS, HEAO, Kweekschool, Sociale of Pedagogische Academie)] | 0.15 | 0.191 | 0.785 | 0.432 |  | -0.224 | 0.523 |
| CDR Env | C(dem_education)[T.Lager beroepsonderwijs (VMBO-B, VMBO-K, praktijkonderwijs, VGLO, LAVO, LTS, Huishoudschool)] | 0.018 | 0.201 | 0.087 | 0.931 |  | -0.376 | 0.411 |
| CDR Env | C(dem_education)[T.Middelbaar algemeen beroepsonderwijs (MBO, MTS)] | 0.027 | 0.191 | 0.14 | 0.888 |  | -0.347 | 0.401 |
| CDR Env | C(dem_education)[T.Middelbaar algemeen voortgezet onderwijs (VMBO-T, MAVO, MULO, 3-jarige HBS)] | -0.004 | 0.198 | -0.018 | 0.986 |  | -0.391 | 0.384 |
| CDR Env | C(dem_education)[T.Universitaire Bachelor (BA, BSc)] | 0.406 | 0.197 | 2.067 | 0.039 | * | 0.021 | 0.792 |
| CDR Env | C(dem_education)[T.Universitaire Master (drs., MA, MSc, mr., ir)] | 0.24 | 0.195 | 1.229 | 0.219 |  | -0.143 | 0.623 |
| CDR Env | C(dem_employment)[T.Ik ben arbeidsongeschikt] | 0.28 | 0.197 | 1.423 | 0.155 |  | -0.106 | 0.667 |
| CDR Env | C(dem_employment)[T.Ik ben met (vervroegd) pensioen] | 0.091 | 0.277 | 0.328 | 0.743 |  | -0.452 | 0.633 |
| CDR Env | C(dem_employment)[T.Ik ben student/scholier  ] | 0.289 | 0.232 | 1.245 | 0.213 |  | -0.166 | 0.744 |
| CDR Env | C(dem_employment)[T.Ik ben uitzendkracht / oproepkracht / invalkracht] | 0.314 | 0.243 | 1.291 | 0.197 |  | -0.163 | 0.789 |
| CDR Env | C(dem_employment)[T.Ik ben werkloos] | 0.418 | 0.212 | 1.976 | 0.048 | * | 0.003 | 0.833 |
| CDR Env | C(dem_employment)[T.Ik ben zelfstandig ondernemer / zzp’er] | 0.112 | 0.197 | 0.567 | 0.571 |  | -0.274 | 0.497 |
| CDR Env | C(dem_employment)[T.Ik werk in loondienst (vast of tijdelijk contract)] | 0.145 | 0.181 | 0.799 | 0.424 |  | -0.21 | 0.499 |
| CDR Env | digiQ_prop | 1.476 | 0.077 | 19.05 | 0 | *** | 1.324 | 1.628 |
| CDR Env | dem_political_lr | -0.036 | 0.011 | -3.181 | 0.002 | ** | -0.058 | -0.014 |
| CDR Soc | Intercept | 3.925 | 0.225 | 17.411 | 0 | *** | 3.483 | 4.367 |
| CDR Soc | C(dem_age_band)[T.25–34] | 0.198 | 0.081 | 2.455 | 0.014 | * | 0.04 | 0.357 |
| CDR Soc | C(dem_age_band)[T.35–44] | 0.315 | 0.084 | 3.769 | 0 | *** | 0.151 | 0.479 |
| CDR Soc | C(dem_age_band)[T.45–54] | 0.313 | 0.083 | 3.776 | 0 | *** | 0.15 | 0.475 |
| CDR Soc | C(dem_age_band)[T.55–65] | 0.604 | 0.085 | 7.126 | 0 | *** | 0.438 | 0.77 |
| CDR Soc | C(dem_gender)[T.Non-binair] | 0.596 | 0.475 | 1.255 | 0.209 |  | -0.335 | 1.526 |
| CDR Soc | C(dem_gender)[T.Vrouw] | 0.176 | 0.043 | 4.092 | 0 | *** | 0.092 | 0.26 |
| CDR Soc | C(dem_gender)[T.Zeg ik liever niet] | 0.471 | 0.565 | 0.833 | 0.405 |  | -0.637 | 1.579 |
| CDR Soc | C(dem_education)[T.Gepromoveerd (dr.)] | 0.343 | 0.269 | 1.273 | 0.203 |  | -0.185 | 0.871 |
| CDR Soc | C(dem_education)[T.Hoger algemeen en voorbereidend wetenschappelijk onderwijs (HAVO, VWO, HBS, Gymnasium, Atheneum)] | 0.16 | 0.193 | 0.832 | 0.406 |  | -0.217 | 0.538 |
| CDR Soc | C(dem_education)[T.Hoger beroepsonderwijs (HBO, HTS, HEAO, Kweekschool, Sociale of Pedagogische Academie)] | 0.224 | 0.178 | 1.258 | 0.208 |  | -0.125 | 0.572 |
| CDR Soc | C(dem_education)[T.Lager beroepsonderwijs (VMBO-B, VMBO-K, praktijkonderwijs, VGLO, LAVO, LTS, Huishoudschool)] | 0.101 | 0.189 | 0.531 | 0.595 |  | -0.27 | 0.471 |
| CDR Soc | C(dem_education)[T.Middelbaar algemeen beroepsonderwijs (MBO, MTS)] | 0.147 | 0.178 | 0.826 | 0.409 |  | -0.202 | 0.496 |
| CDR Soc | C(dem_education)[T.Middelbaar algemeen voortgezet onderwijs (VMBO-T, MAVO, MULO, 3-jarige HBS)] | 0.018 | 0.183 | 0.097 | 0.922 |  | -0.341 | 0.377 |
| CDR Soc | C(dem_education)[T.Universitaire Bachelor (BA, BSc)] | 0.32 | 0.183 | 1.754 | 0.08 |  | -0.038 | 0.678 |
| CDR Soc | C(dem_education)[T.Universitaire Master (drs., MA, MSc, mr., ir)] | 0.191 | 0.184 | 1.042 | 0.297 |  | -0.169 | 0.551 |
| CDR Soc | C(dem_employment)[T.Ik ben arbeidsongeschikt] | -0.042 | 0.149 | -0.281 | 0.779 |  | -0.334 | 0.25 |
| CDR Soc | C(dem_employment)[T.Ik ben met (vervroegd) pensioen] | -0.186 | 0.195 | -0.957 | 0.338 |  | -0.568 | 0.195 |
| CDR Soc | C(dem_employment)[T.Ik ben student/scholier  ] | 0.021 | 0.172 | 0.12 | 0.904 |  | -0.317 | 0.359 |
| CDR Soc | C(dem_employment)[T.Ik ben uitzendkracht / oproepkracht / invalkracht] | 0.092 | 0.221 | 0.417 | 0.677 |  | -0.341 | 0.526 |
| CDR Soc | C(dem_employment)[T.Ik ben werkloos] | 0.1 | 0.16 | 0.629 | 0.529 |  | -0.212 | 0.413 |
| CDR Soc | C(dem_employment)[T.Ik ben zelfstandig ondernemer / zzp’er] | -0.142 | 0.146 | -0.978 | 0.328 |  | -0.428 | 0.143 |
| CDR Soc | C(dem_employment)[T.Ik werk in loondienst (vast of tijdelijk contract)] | -0.122 | 0.127 | -0.962 | 0.336 |  | -0.371 | 0.127 |
| CDR Soc | digiQ_prop | 1.469 | 0.069 | 21.383 | 0 | *** | 1.335 | 1.604 |
| CDR Soc | dem_political_lr | -0.008 | 0.01 | -0.813 | 0.416 |  | -0.027 | 0.011 |
| CDR Gov | Intercept | 3.843 | 0.242 | 15.901 | 0 | *** | 3.369 | 4.317 |
| CDR Gov | C(dem_age_band)[T.25–34] | 0.125 | 0.084 | 1.49 | 0.136 |  | -0.04 | 0.29 |
| CDR Gov | C(dem_age_band)[T.35–44] | 0.263 | 0.087 | 3.031 | 0.002 | ** | 0.093 | 0.434 |
| CDR Gov | C(dem_age_band)[T.45–54] | 0.276 | 0.087 | 3.182 | 0.002 | ** | 0.106 | 0.446 |
| CDR Gov | C(dem_age_band)[T.55–65] | 0.624 | 0.087 | 7.179 | 0 | *** | 0.453 | 0.794 |
| CDR Gov | C(dem_gender)[T.Non-binair] | 0.424 | 0.485 | 0.874 | 0.382 |  | -0.527 | 1.375 |
| CDR Gov | C(dem_gender)[T.Vrouw] | 0.162 | 0.044 | 3.677 | 0 | *** | 0.076 | 0.248 |
| CDR Gov | C(dem_gender)[T.Zeg ik liever niet] | 0.112 | 0.439 | 0.255 | 0.798 |  | -0.748 | 0.972 |
| CDR Gov | C(dem_education)[T.Gepromoveerd (dr.)] | 0.744 | 0.238 | 3.117 | 0.002 | ** | 0.276 | 1.211 |
| CDR Gov | C(dem_education)[T.Hoger algemeen en voorbereidend wetenschappelijk onderwijs (HAVO, VWO, HBS, Gymnasium, Atheneum)] | 0.388 | 0.206 | 1.885 | 0.059 |  | -0.015 | 0.792 |
| CDR Gov | C(dem_education)[T.Hoger beroepsonderwijs (HBO, HTS, HEAO, Kweekschool, Sociale of Pedagogische Academie)] | 0.457 | 0.193 | 2.361 | 0.018 | * | 0.078 | 0.835 |
| CDR Gov | C(dem_education)[T.Lager beroepsonderwijs (VMBO-B, VMBO-K, praktijkonderwijs, VGLO, LAVO, LTS, Huishoudschool)] | 0.179 | 0.204 | 0.879 | 0.379 |  | -0.22 | 0.579 |
| CDR Gov | C(dem_education)[T.Middelbaar algemeen beroepsonderwijs (MBO, MTS)] | 0.26 | 0.194 | 1.339 | 0.18 |  | -0.121 | 0.641 |
| CDR Gov | C(dem_education)[T.Middelbaar algemeen voortgezet onderwijs (VMBO-T, MAVO, MULO, 3-jarige HBS)] | 0.142 | 0.199 | 0.712 | 0.476 |  | -0.249 | 0.532 |
| CDR Gov | C(dem_education)[T.Universitaire Bachelor (BA, BSc)] | 0.545 | 0.198 | 2.758 | 0.006 | ** | 0.158 | 0.933 |
| CDR Gov | C(dem_education)[T.Universitaire Master (drs., MA, MSc, mr., ir)] | 0.476 | 0.2 | 2.385 | 0.017 | * | 0.085 | 0.867 |
| CDR Gov | C(dem_employment)[T.Ik ben arbeidsongeschikt] | -0.046 | 0.156 | -0.296 | 0.767 |  | -0.352 | 0.26 |
| CDR Gov | C(dem_employment)[T.Ik ben met (vervroegd) pensioen] | -0.022 | 0.2 | -0.112 | 0.911 |  | -0.415 | 0.37 |
| CDR Gov | C(dem_employment)[T.Ik ben student/scholier  ] | 0.024 | 0.185 | 0.132 | 0.895 |  | -0.339 | 0.387 |
| CDR Gov | C(dem_employment)[T.Ik ben uitzendkracht / oproepkracht / invalkracht] | -0.068 | 0.218 | -0.312 | 0.755 |  | -0.496 | 0.36 |
| CDR Gov | C(dem_employment)[T.Ik ben werkloos] | 0.129 | 0.164 | 0.787 | 0.431 |  | -0.193 | 0.452 |
| CDR Gov | C(dem_employment)[T.Ik ben zelfstandig ondernemer / zzp’er] | -0.164 | 0.152 | -1.076 | 0.282 |  | -0.462 | 0.135 |
| CDR Gov | C(dem_employment)[T.Ik werk in loondienst (vast of tijdelijk contract)] | -0.177 | 0.136 | -1.297 | 0.195 |  | -0.444 | 0.09 |
| CDR Gov | digiQ_prop | 1.77 | 0.07 | 25.226 | 0 | *** | 1.632 | 1.907 |
| CDR Gov | dem_political_lr | -0.004 | 0.01 | -0.431 | 0.667 |  | -0.024 | 0.015 |

<!-- ⤷ 06_responsibility_attribution.md -->

# 🧭 Responsibility Attributions

## RQ1 — Environment: perceived responsibility (higher = more responsibility)

| actor_item | mean | rank |
|:--|--:|--:|
| responsibility attri_1 | 5.168 | 1 |
| responsibility attri_5 | 5.07 | 2 |
| responsibility attri_4 | 5.032 | 3 |
| responsibility attri_2 | 4.878 | 4 |
| responsibility attri_7 | 4.364 | 5 |
| responsibility attri_8 | 4.344 | 6 |
| responsibility attri_6 | 4.315 | 7 |
| responsibility attri_3 | 4.114 | 8 |

## RQ1 — Social: perceived responsibility

| actor_item | mean | rank |
|:--|--:|--:|
| resp_attribut_societ_1 | 5.134 | 1 |
| resp_attribut_societ_5 | 5.124 | 2 |
| resp_attribut_societ_4 | 5.111 | 3 |
| resp_attribut_societ_2 | 4.928 | 4 |
| resp_attribut_societ_8 | 4.521 | 5 |
| resp_attribut_societ_7 | 4.477 | 6 |
| resp_attribut_societ_6 | 4.375 | 7 |
| resp_attribut_societ_3 | 4.307 | 8 |

## RQ1 — Governance/Ethical use: perceived responsibility

| actor_item | mean | rank |
|:--|--:|--:|
| resp_attribut_regula_4 | 5.308 | 1 |
| resp_attribut_regula_5 | 5.3 | 2 |
| resp_attribut_regula_1 | 5.033 | 3 |
| resp_attribut_regula_2 | 4.776 | 4 |
| resp_attribut_regula_7 | 4.48 | 5 |
| resp_attribut_regula_8 | 4.441 | 6 |
| resp_attribut_regula_3 | 4.17 | 7 |
| resp_attribut_regula_6 | 4.114 | 8 |

<!-- ⤷ 07_prereg_scaffold.md -->


# 📝 Preregistration scaffold (draft)

## Outcomes
- **CDR_total_mean** (primary), **CDR_env_mean**, **CDR_soc_mean**, **CDR_gov_mean** (secondary).

## Predictors & Covariates
- Predictors: env_consc, soc_consc, trust_mean, digiQ_prop.
- Covariates: dem_age_band, dem_gender, dem_education, dem_employment, dem_political_lr (as available).

## Reliability
| Domein | k | N_alpha | Cronbach_α |
|:--|--:|:--|:--|
| Environmental | 3 |  |  |
| Social | 9 |  |  |
| Governance | 8 |  |  |

## Models
- **M1 (Primary):** CDR_total_mean ~ z(env_consc) + z(soc_consc) + z(trust_mean) + z(digiQ_prop) + demographics.
- **M2–M4:** Same predictors with CDR_env_mean, CDR_soc_mean, CDR_gov_mean.

(Where z(x) denotes standardization; in the quick results above we reported unstandardized β with HC3 SE.)

## Missing data
- Listwise deletion within each model. No imputation for prereg primary; sensitivity analyses optional.

## Responsibility attribution (RQ1)
- Report actor-level means within each domain; interpret rank order patterns.


---

# 🔎 Individual Sections

---

### ⤷ 01_sample_overview.md


...

# 🧭 Responsibility Attributions

## RQ1 — Environment: perceived responsibility (higher = more responsibility)

| actor_item | mean | rank |
|:--|--:|--:|
| responsibility attri_1 | 5.168 | 1 |
| responsibility attri_5 | 5.07 | 2 |
| responsibility attri_4 | 5.032 | 3 |
| responsibility attri_2 | 4.878 | 4 |
| responsibility attri_7 | 4.364 | 5 |
| responsibility attri_8 | 4.344 | 6 |
| responsibility attri_6 | 4.315 | 7 |
| responsibility attri_3 | 4.114 | 8 |

## RQ1 — Social: perceived responsibility

| actor_item | mean | rank |
|:--|--:|--:|
| resp_attribut_societ_1 | 5.134 | 1 |
| resp_attribut_societ_5 | 5.124 | 2 |
| resp_attribut_societ_4 | 5.111 | 3 |
| resp_attribut_societ_2 | 4.928 | 4 |
| resp_attribut_societ_8 | 4.521 | 5 |
| resp_attribut_societ_7 | 4.477 | 6 |
| resp_attribut_societ_6 | 4.375 | 7 |
| resp_attribut_societ_3 | 4.307 | 8 |

## RQ1 — Governance/Ethical use: perceived responsibility

| actor_item | mean | rank |
|:--|--:|--:|
| resp_attribut_regula_4 | 5.308 | 1 |
| resp_attribut_regula_5 | 5.3 | 2 |
| resp_attribut_regula_1 | 5.033 | 3 |
| resp_attribut_regula_2 | 4.776 | 4 |
| resp_attribut_regula_7 | 4.48 | 5 |
| resp_attribut_regula_8 | 4.441 | 6 |
| resp_attribut_regula_3 | 4.17 | 7 |
| resp_attribut_regula_6 | 4.114 | 8 |

---

### ⤷ 07_prereg_scaffold.md



# 📝 Preregistration scaffold (draft)

## Outcomes
- **CDR_total_mean** (primary), **CDR_env_mean**, **CDR_soc_mean**, **CDR_gov_mean** (secondary).

## Predictors & Covariates
- Predictors: env_consc, soc_consc, trust_mean, digiQ_prop.
- Covariates: dem_age_band, dem_gender, dem_education, dem_employment, dem_political_lr (as available).

## Reliability
| Domein | k | N_alpha | Cronbach_α |
|:--|--:|:--|:--|
| Environmental | 3 |  |  |
| Social | 9 |  |  |
| Governance | 8 |  |  |

## Models
- **M1 (Primary):** CDR_total_mean ~ z(env_consc) + z(soc_consc) + z(trust_mean) + z(digiQ_prop) + demographics.
- **M2–M4:** Same predictors with CDR_env_mean, CDR_soc_mean, CDR_gov_mean.

(Where z(x) denotes standardization; in the quick results above we reported unstandardized β with HC3 SE.)

## Missing data
- Listwise deletion within each model. No imputation for prereg primary; sensitivity analyses optional.

## Responsibility attribution (RQ1)
- Report actor-level means within each domain; interpret rank order patterns.


### 📊 Correlations (DataFrame view)

ModuleNotFoundError: No module named 'caas_jupyter_tools'

In [32]:
df['Age '].value_counts()

Age 
55 - 65 jaar    540
45 - 54 jaar    520
25 - 34 jaar    497
35 - 44 jaar    447
18 - 24 jaar    239
Name: count, dtype: int64